In [1]:
#install dependencies
!pip install requests pandas
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#imports
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import requests
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [4]:
#get data
%cd /content/drive/MyDrive/Colab Notebooks/F1dataset

circuits = pd.read_csv('circuits.csv')
drivers = pd.read_csv('drivers.csv')
lap_times = pd.read_csv('lap_times.csv')
pit_stops = pd.read_csv('pit_stops.csv')
qualifying = pd.read_csv('qualifying.csv')
results = pd.read_csv('results.csv')
seasons = pd.read_csv('seasons.csv')
status = pd.read_csv('status.csv')

#Explore the datasets
datasets = {
    "Circuits": circuits,
    "Drivers": drivers,
    "Lap Times": lap_times,
    "Pit Stops": pit_stops,
    "Qualifying": qualifying,
    "Results": results,
    "Seasons": seasons,
    "Status": status
}

/content/drive/MyDrive/Colab Notebooks/F1dataset


In [26]:
#data merging
lap_times = datasets['Lap Times']
drivers = datasets['Drivers']
circuits = datasets['Circuits']
qualifying = datasets['Qualifying']

merged_data = lap_times.merge(drivers, on='driverId', how='inner')
merged_data = merged_data.merge(circuits, left_on='driverId', right_on='circuitId', how='inner')

merged_data.head()

,raceId,driverId,lap,position,time,milliseconds,driverRef,number,code,forename,...,url_x,circuitId,circuitRef,name,location,country,lat,lng,alt,url_y
0,841,20,1,1,1:38.109,98109,vettel,5,VET,Sebastian,...,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,nurburgring,Nürburgring,Nürburg,Germany,50.3356,6.9475,578,http://en.wikipedia.org/wiki/N%C3%BCrburgring
1,841,20,2,1,1:33.006,93006,vettel,5,VET,Sebastian,...,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,nurburgring,Nürburgring,Nürburg,Germany,50.3356,6.9475,578,http://en.wikipedia.org/wiki/N%C3%BCrburgring
2,841,20,3,1,1:32.713,92713,vettel,5,VET,Sebastian,...,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,nurburgring,Nürburgring,Nürburg,Germany,50.3356,6.9475,578,http://en.wikipedia.org/wiki/N%C3%BCrburgring
3,841,20,4,1,1:32.803,92803,vettel,5,VET,Sebastian,...,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,nurburgring,Nürburgring,Nürburg,Germany,50.3356,6.9475,578,http://en.wikipedia.org/wiki/N%C3%BCrburgring
4,841,20,5,1,1:32.342,92342,vettel,5,VET,Sebastian,...,http://en.wikipedia.org/wiki/Sebastian_Vettel,20,nurburgring,Nürburgring,Nürburg,Germany,50.3356,6.9475,578,http://en.wikipedia.org/wiki/N%C3%BCrburgring


In [7]:
#load the data
torch.utils.data.Dataset
torch.utils.data.DataLoader(merged_data)

In [32]:
#Split the data
featured_columns = ['driverId', 'circuitId', 'lap', 'milliseconds']
X = merged_data[featured_columns]
y = merged_data['milliseconds']

categorical_features = ['driverId', 'circuitId']
numerical_features = ['lap', 'time', 'milliseconds']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Split data into training and testing sets

In [33]:
#data to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [45]:
#defining model
class F1Predictor(nn.Module):
  def __init__(self):
    super(F1Predictor, self).__init__()
    self.fc1 = nn.Linear(4, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 1)
    self.relu = nn.ReLU()

  def forward(self, x): # Corrected indentation
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [46]:
#initialize model
input = X_train_tensor
model = F1Predictor()
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

#data loaders
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
#training
num_epochs = 10
for epoch in range(num_epochs):
  model.train()
  for x, y in train_loader:
    optimizer.zero_grad()
    outputs = model(x)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

  if (epoch + 1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
  test_outputs = model(X_test_tensor)
  test_loss = criterion(test_outputs, y_test_tensor)
  print(f'Test Loss: {test_loss.item():.4f}')

Epoch [10/10], Loss: 317055936.0000


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([68605])) that is different to the input size (torch.Size([68605, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
